# Using Municipal Open Data to Analyze Councillor Voting Patterns
### by Mark M<sup>c</sup>Carthy

I find myself in a position where my previous work is proprietary to my former employers, and I don't have much of a publicly available portfolio. To fix this, I wanted to find a sizeable public dataset then perform a non-trivial analysis, and to have fun doing it! In this project I used attendance data from the city councils of Toronto to study councillor absenteeism, and you can read about this in [another notebook](). I later wrote a clustering algorithm to group councillors based their voting records, developed it with Toronto data and evaluated it with Calgary data, and you can see the analysis below. 

The clustering algorithm was a heirarchichal agglomerative weight-based non-euclidean clustering using the sum of the variances in the clusters as the objective function to be minimized. A complex imputation was also performed on sparse, non-euclidean data with highly non-uniform uncertainties, with the intent of optimizing the result of the clustering to come.

Many municipalities have open source data available on a very wide number of topics. For example, in Toronto you can find data on everything from all public city-provided wifi, to red-light camera locations, to restaurant food inspection results. Here are the data for [Toronto](https://open.toronto.ca/) and [Calgary](https://data.calgary.ca/).

To get started, let's get all the modules and packages we'll need to analyze the data. Since the datasets were likely to not be overly large, I used Pandas throughout this project.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
print('pandas version = '+pd.__version__)
print('numpy version = '+np.__version__)

pandas version = 2.0.3
numpy version = 1.24.3


Next we need to grab all the data. To keep it simple, I downloaded it all to a local directory and concatenated the different datafiles into one big table. The column names changed at some point to be more human-readable, from names like `SessionType` to `Session Type`,  but thankfully this was the only way in which they changed, so I could just force a consistent column name upon ingestion. There's a column `_id` which is clearly meant as an index which counts out the entries in each data set, but since I was concatenating different data sets together, I was just going to have to use my own index and ignore this column.

In [2]:
# need to grab attendance data for Toronto
dir_name = '/Users/markmccarthy/Documents/coding_fun/toronto_dsci/councillors/'
short_filenames_attendance = [
    'councillors-meeting-attendance-2022-2026.csv',
    'councillors-meeting-attendance-2018-2022.csv',
    'councillors-meeting-attendance-2014-2018.csv',
    'councillors-meeting-attendance-2010-2014.csv',
    'councillors-meeting-attendance-2006-2010.csv'
]
filenames_attendance = [dir_name + f for f in short_filenames_attendance] 

In [3]:
#The columns are consistent but the names are not. Let's fix that
col_names = ['_id', 'Term', 'First Name', 'Last Name', 'Committee', 'MTG #', 'Session Date', 'Session Type', 'Session Start-End Time', 'Present']

raw_attendance_df = pd.concat((pd.read_csv(f, header=0, names=col_names) for f in filenames_attendance), ignore_index=True)

A quick look at the data to get a feel for what we're working with...

In [4]:
raw_attendance_df.head()

,_id,Term,First Name,Last Name,Committee,MTG #,Session Date,Session Type,Session Start-End Time,Present
0,1,2022-2026,Paula,Fletcher,CreateTO,1,2022-11-18,Morning,09:31AM - 09:50AM,N
1,2,2022-2026,Paul,Ainslie,City Council,1,2022-11-23,Afternoon,14:03PM - 15:25PM,Y
2,3,2022-2026,Brad,Bradford,City Council,1,2022-11-23,Afternoon,14:03PM - 15:25PM,Y
3,4,2022-2026,Alejandra,Bravo,City Council,1,2022-11-23,Afternoon,14:03PM - 15:25PM,Y
4,5,2022-2026,Jon,Burnside,City Council,1,2022-11-23,Afternoon,14:03PM - 15:25PM,Y


In [5]:
raw_attendance_df.describe(include = ['object'])

,Term,First Name,Last Name,Committee,Session Date,Session Type,Session Start-End Time,Present
count,86530,86530,86530,86530,86530,86530,86530,86530
unique,5,68,79,124,2178,3,5787,2
top,2010-2014,John,Crawford,City Council,2011-06-15,Afternoon,09:40AM - 12:30PM,Y
freq,31894,4878,2677,53409,495,38702,785,74956


In [6]:
# check if there is any missing data
raw_attendance_df.isnull().any()

_id                       False
Term                      False
First Name                False
Last Name                 False
Committee                 False
MTG #                     False
Session Date              False
Session Type              False
Session Start-End Time    False
Present                   False
dtype: bool

Clearly each row is the attendance record for one councillor at one meeting. Mayors seem to be included in these records as if they were just another councillor, so we shouldn't need to provide any special treatment to deal with their data. Now it's time to transform our dataframe in a way that lets us ask meaningful questions of it.<br>
First, I grouped the attendance records for each councillor together so we can get how often they attended meetings to which they were eligible to attend. I kept only city council meetings, since they represent 53k of our 86k entries, and some of these committees are going to be difficult to compare to an ordinary council meeting.

In [7]:
# Make a copy to not alter the raw data
attend_df = raw_attendance_df.copy()

# Remove committee meetings, keep only council meetings
attend_df = attend_df.loc[attend_df['Committee']=='City Council']

# If we're identifying councillors by name, we should put it in one single column for convenience 
attend_df['Name'] = attend_df['First Name'] + ' ' + attend_df['Last Name']
attend_df = attend_df.drop(columns = ['First Name','Last Name'])

# Converting 'Y' or 'N' into 1 or 0 is a handy trick to simplify this sort of data
attend_df['Present'] = np.where(attend_df['Present']=='Y', 1, 0)
attend_df = attend_df.groupby('Name')['Present'].agg(['count', 'sum'])
attend_df = attend_df.rename(columns={'count': 'TimesElegibleToAttend', 'sum': 'TimesPresent'}).reset_index()

# This is the column we've all been waiting for, how often did they attend meetings?
attend_df['AttendancePercent'] = 100 * attend_df['TimesPresent'] / attend_df['TimesElegibleToAttend']

display(attend_df.sort_values(by=['AttendancePercent'],ascending=True))

,Name,TimesElegibleToAttend,TimesPresent,AttendancePercent
27,Giorgio Mammoliti,1027,613,59.688413
76,Ron Moeser,831,515,61.973526
18,David Miller,154,100,64.935065
64,Mike Feldman,154,101,65.584416
61,Michelle Holland,286,191,66.783217
...,...,...,...,...
23,Doug Holyday,479,471,98.329854
30,Gord Perks,1340,1322,98.656716
80,Stephen Holyday,707,704,99.575672
3,Alejandra Bravo,57,57,100.000000


Our most reliable councillor is recently elected mayor Olivia Chow! ... with only 15 possible meetings so far. To make a more fair comparison of our dataset we should only include those who have been to a reasonable number of meetings  

In [8]:
display(attend_df.TimesElegibleToAttend.describe())

count      83.000000
mean      643.481928
std       437.994274
min        11.000000
25%       154.000000
50%       633.000000
75%      1027.000000
max      1340.000000
Name: TimesElegibleToAttend, dtype: float64

In [9]:
# We can use the 25th percentile as a cutoff, because it's reasonably large and leaves us with plenty of remaining councillors
attend_df = attend_df.loc[attend_df['TimesElegibleToAttend']>154].sort_values(by=['AttendancePercent'],ascending=True)
display(attend_df)
display(attend_df.AttendancePercent.describe())

,Name,TimesElegibleToAttend,TimesPresent,AttendancePercent
27,Giorgio Mammoliti,1027,613,59.688413
76,Ron Moeser,831,515,61.973526
61,Michelle Holland,286,191,66.783217
75,Rob Ford,751,527,70.173103
35,Jaye Robinson,1186,900,75.885329
...,...,...,...,...
78,Sarah Doucette,873,846,96.907216
24,Frances Nunziata,1340,1304,97.313433
23,Doug Holyday,479,471,98.329854
30,Gord Perks,1340,1322,98.656716


count    61.000000
mean     87.138054
std       8.429293
min      59.688413
25%      83.242401
50%      88.467615
75%      93.799823
max      99.575672
Name: AttendancePercent, dtype: float64

---

After removing the low sample size councillors, we arrive at an average (mean) attendance of ~87%. Some councillors have attended almost every meeting they were able to (looking at you, Stephen Holyday!), whereas some are missing a quarter or more! Our anecdote about Rob Ford seems to hold up, only 3 councillors attended fewer meetings. Some quick googling helps to exonerate one of them, Ron Moeser, who suffered from cancer and eventually succumbed while in office.<br>

The attendance data doesn't offer us too many more insights, but it does give us one more tidbit when we group by `Session Type`:

In [10]:
# Make a copy to not alter the raw data
attend_df2 = raw_attendance_df.copy()

# Remove committee meetings, keep only council meetings
attend_df2 = attend_df2.loc[attend_df2['Committee']=='City Council']

# Converting 'Y' or 'N' into 1 or 0 is a handy trick to simplify this sort of data
attend_df2['Present'] = np.where(attend_df2['Present']=='Y', 1, 0)
attend_df2 = attend_df2.groupby('Session Type')['Present'].agg(['count', 'sum'])
attend_df2 = attend_df2.rename(columns={'count': 'TimesElegibleToAttend', 'sum': 'TimesPresent'}).reset_index()

# This is the column we've all been waiting for, how often did they attend meetings?
attend_df2['AttendancePercent'] = 100 * attend_df2['TimesPresent'] / attend_df2['TimesElegibleToAttend']

display(attend_df2.sort_values(by=['AttendancePercent'],ascending=True))

,Session Type,TimesElegibleToAttend,TimesPresent,AttendancePercent
1,Evening,9441,7687,81.421460
0,Afternoon,24806,21431,86.394421
2,Morning,19162,17138,89.437428


Councillors are early-birds, not night-owls!

---

## Councillor clustering based on voting behaviour

Now that we've grown comfortable with the open data, let's see if we can do something more sophisticated. If we look at voting records rather than just attendance records, we can try to group councillors together based on how similar they are from each other. To see if we've done a good job clustering, I built everything that follows using the Toronto council voting data. When I was satisfied that the results looked reasonable, I tested to see if it worked using the Calgary council voting data.<br>

Unfortunately we see that there is a substantial difference between the two datasets. Toronto's voting records go back to 2006, whereas Calgary's only go back to 2017. To go back further with Calgary's data, it looks like I'd have to do some pdf scraping of the meeting minutes. Since this is supposed to be a ***fun*** project, I'll just let it be. If I were to do this again, I'd try to find a better train-validation-test breakdown.

### Data ingestion and cleanup

First we need to import our modules and ingest the data. I've decided to duplicate all our dataframes into two sets, a Toronto set and a Calgary set. Had this been a bigger project, I'd love to turn all these into function calls or some kind of sklearn pipelines, but that's beyond the scope for now.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
print('pandas version = '+pd.__version__)
print('numpy version = '+np.__version__)

pandas version = 2.0.3
numpy version = 1.24.3


In [12]:
# Let's label our two datasets, in case I forget my naming scheme
city_dict = {0:'Toronto',1:'Calgary'}

dir_name = '/Users/markmccarthy/Documents/coding_fun/toronto_dsci/councillors/'
filenames = {}

# Toronto data 
short_filenames = [
    'member-voting-record-2022-2026.csv',
    'member-voting-record-2018-2022.csv',
    'member-voting-record-2014-2018.csv',
    'member-voting-record-2010-2014.csv',
    'member-voting-record-2006-2010.csv'
]
filenames[0] = [dir_name + f for f in short_filenames] 

#Calgary data
filenames[1] = [dir_name + 'Council_and_Committee_Votes_20231024.csv']

In [ ]:
raw_voting_df = {}

# Toronto
raw_voting_df[0] = pd.concat((pd.read_csv(f, index_col = '_id') for f in filenames[0]), ignore_index=True)

#Calgary
raw_voting_df[1] = pd.read_csv(filenames[1][0])

In [ ]:
# Get a sense of what the data looks like
for city_id, df in raw_voting_df.items():
    print(f'City = {city_dict[city_id]}')
    display(df.head())
    
for city_id, df in raw_voting_df.items():
    print(f'City = {city_dict[city_id]}')
    display(df.describe(include='all'))

In [ ]:
# Check for duplicated data
for city_id, df in raw_voting_df.items():
    print(f'City = {city_dict[city_id]}')
    print(f'Number of duplicated rows = {df[df.duplicated()].shape[0]}')
    print('') # could use more whitespace

In [ ]:
# Check for missing data
for city_id, df in raw_voting_df.items():
    print(f'City = {city_dict[city_id]}')
    display(df.isnull().sum())
    print('') # could use more whitespace

We can see that the Toronto and Calgary data have a lot in common. Both have varied `Committee` or `MeetingType` including council votes and committee votes, and each row represents each councillor at each vote. We can already see some features of the data, for example, the Toronto `Date/Time` column is mostly empty, so we should be careful if ever using it.

In [ ]:
for city_id, df in raw_voting_df.items():
    print(f'City = {city_dict[city_id]}')
    # it's quite convenient that the column name 'Vote' appears in both datasets!
    display(df.Vote.value_counts())
    print('')

One major difference between the datasets is apparent here... In Toronto absent councillors are always given a row in the data where they `Vote` `Absent`, whereas the absent councillors in Calgary are omitted from the table entirely, with rare exceptions.<br><br>
This feature lets us quickly redo our attendance analysis in Toronto quite easily, but on a per-vote rather than per-meeting basis! To do so in Calgary, however, would require ingesting some new data that tells us who was a councillor and when. We could then impute the `Absent`s ourselves, but there may be many situations where councillors are not on a committee or otherwise not eligible to vote, which could cause problems.<br><br>
So, for now, let's try to quickly redo our attendance analysis on a per-vote basis and see if we get similar results!

In [ ]:
# Check which councillors missed votes, Toronto only
voting_attend_df = raw_voting_df[0].copy()

# Get rid of committee votes 
voting_attend_df = voting_attend_df.loc[voting_attend_df['Committee'] == 'City Council']

# Group the names into one column (this isn't strictly necessary)
voting_attend_df['Name'] = voting_attend_df['First Name'] + ' ' + voting_attend_df['Last Name']
voting_attend_df = voting_attend_df.drop(columns = ['First Name','Last Name'])

# Group by councillor and get the percentage of absent votes
voting_attend_df['Absent'] = np.where(voting_attend_df['Vote']=='Absent', 1, 0)
voting_attend_df = voting_attend_df.groupby('Name')['Absent'].agg(['count', 'sum'])
voting_attend_df['AbsentPercent'] = 100 * voting_attend_df['sum'] / voting_attend_df['count']
voting_attend_df = voting_attend_df.rename(columns={'count': 'EligibleVotes', 'sum': 'TimesAbsent'}).reset_index()

# Remove councillors with a low vote count
# There are probably better choices for a cut-off, but the 25th percentile is at least consistent with the previous analysis
display(voting_attend_df.EligibleVotes.describe())
voting_attend_df = voting_attend_df.loc[voting_attend_df['EligibleVotes'] > voting_attend_df.EligibleVotes.quantile(0.25)]

# Display results
display(voting_attend_df.sort_values(by=['AbsentPercent'],ascending=False))

Looks good, a lot of the same names show up as before! The per-meeting numbers are probably more meaningful than the per-vote numbers because meeting attendance is more likely to be independent data points, but it's nice to see some relative agreement.<br>
<br>
Next, we can look at one of the remaining unexamined columns in the Toronto data, the `Motion Type`. I can imagine that some motions are controversial, some motions are skippable, and some are good reflections of a candidates viewpoints. Let's see how it breaks down:

In [ ]:
# Check which Motion Types are controversial and/or commonly skipped
voting_motion_type_df = raw_voting_df[0].copy()

# Get the total votes of each response grouped by Motion Type
voting_motion_type_df = voting_motion_type_df.groupby('Motion Type').apply(lambda x: pd.Series(dict(
    yes=(x.Vote=='Yes').sum(),
    no=(x.Vote=='No').sum(),
    absent=(x.Vote=='Absent').sum()
)))

# Now let's calculate the secondary numbers
voting_motion_type_df['Total'] = voting_motion_type_df['yes'] + voting_motion_type_df['no'] + voting_motion_type_df['absent']

voting_motion_type_df['Yes %']    = 100 * voting_motion_type_df['yes'] / voting_motion_type_df['Total']
voting_motion_type_df['No %']     = 100 * voting_motion_type_df['no'] / voting_motion_type_df['Total']
voting_motion_type_df['Absent %'] = 100 * voting_motion_type_df['absent'] / voting_motion_type_df['Total']

# Sort by no%... how controversial are these motion types?
display(voting_motion_type_df[['Total','Yes %','No %','Absent %']].sort_values(by='No %', ascending = False))

Here we reach a difficult decision... should any motion types be removed because they do not actually reflect councillor voting patterns, which is what we want to observe? You could argue that `Adopt Minutes` and `Introduce Report` are so agreeable that they offer little information about our voters, but it's unclear how to define when a motion is meaningless compared to insightful. Even votes on banal-sounding issues like `Recess` could contain information on the political leanings of councillors, so it's tough to justify removing any of these. Perhaps more importantly, it would be far more difficult to remove similar motions from the Calgary data since this information is obscured within the `Resolution` column, and treating the datasets differently would undermine comparisons between the two. 

In light of this, I've chosen to keep all of our motion types. It's something we can keep in mind if we ever revisit this analysis later.

Let's now take a closer look at the Calgary data and see if there is any cleanup required. First we notice some councillor naming inconsistencies:

In [ ]:
display(raw_voting_df[1].loc[raw_voting_df[1]['Voter'].str.contains('Gondek')].Voter.value_counts())
print()
display(raw_voting_df[1].loc[raw_voting_df[1]['Voter'].str.contains('Pootmans')].Voter.value_counts())
print()
display(raw_voting_df[1].loc[raw_voting_df[1]['Voter'].str.contains('Nenshi')].Voter.value_counts())

The time period of our data spans when Jyoti Gondek was a councillor and when she became mayor. However, she remains just one person, not two, so we need to merge her entries. We should probably also change Naheed Nenshi's name to be consistent with all the councillors. Lastly, we ought to fix those two times that John (Richard) Pootmans' name was entered differently.

One interesting feature of Calgary's municipal government is that there are some committees where private citizens are appointed to the committee and will vote along with some councillors. Since we're not including committee votes anyway, this won't be an issue for our clustering later.

In [ ]:
list_of_councillors = raw_voting_df[1].loc[raw_voting_df[1]['MeetingType'].str.contains('Council')].Voter.unique()
display(raw_voting_df[1].loc[raw_voting_df[1]['Voter'].isin(list_of_councillors)==False].MeetingType.value_counts())

Lastly, let's look at those rare `Vote`s that are `Absent` and make sure nothing strange is happening there.

In [ ]:
display(raw_voting_df[1].loc[raw_voting_df[1].Vote =='Absent'].Voter.value_counts())

In [ ]:
# Wait, who is '. .'?
display(raw_voting_df[1].loc[raw_voting_df[1].Voter =='. .'].Vote.value_counts())

In [ ]:
# A 'yes' vote was unattributed???
display(raw_voting_df[1].loc[(raw_voting_df[1].Voter == '. .') & (raw_voting_df[1].Vote == 'Yes')])

In [ ]:
# When did this vote happen? How did it happen?
temp_df = raw_voting_df[1].loc[(raw_voting_df[1].Voter == '. .') & (raw_voting_df[1].Vote == 'Yes')].drop(columns=['Voter','Vote'])
col_list = temp_df.columns.to_list()

display(pd.merge(left = temp_df, right = raw_voting_df[1], on = col_list, how='left'))

I can understand that someone keeping minutes of council meetings might be a bit lazy when entering an `Absent` vote and input a name like `. .`, but inputting a `Yes` vote seems outlandish! Cross-referencing the list of councillors in the meeting and the councillors at the time show that Ray Jones from Ward 10 is missing. So clearly `. .` is Ray Jones, right? Well, Ray Jones resigned from council in October 2020 citing health issues and no by-election was held, so the seat was vacant until the next election in October 2021. This looks like an actual mistake where a vacant seat was recorded as voting `Yes`, but thankfully the result of the vote was not altered by the mistake!

Now that we understand the peculiarities of our data, we can finally clean it up and get ready to cluster!

In [ ]:
# Cleaning raw data:
clus_vote_df = {}

# -----------------------------------------------------------------------------------------------
# Toronto
clus_vote_df[0] = raw_voting_df[0].copy()

# The Date/Time column is mostly empty
clus_vote_df[0] = clus_vote_df[0].drop(columns='Date/Time')

# Only council meetings
clus_vote_df[0] = clus_vote_df[0].loc[clus_vote_df[0].Committee == 'City Council']

# Consolidating the names into one column makes life easier
clus_vote_df[0]['Name'] = clus_vote_df[0]['First Name'] + ' ' + clus_vote_df[0]['Last Name']
clus_vote_df[0] = clus_vote_df[0].drop(columns = ['First Name','Last Name'])

# -----------------------------------------------------------------------------------------------
# Calgary
clus_vote_df[1] = raw_voting_df[1].copy()

# Only council meetings
clus_vote_df[1] = clus_vote_df[1].loc[clus_vote_df[1]['MeetingType'].str.contains('Council')]

# Standardize councillor names
clus_vote_df[1] = clus_vote_df[1].replace({'Mayor J. Gondek':'Jyoti Gondek',
                                     'Mayor N. Nenshi':'Naheed Nenshi',
                                     'Richard Pootmans':'John (Richard) Pootmans'})

# Rename the Voter column to make consistent with the Toronto data
clus_vote_df[1] = clus_vote_df[1].rename(columns={'Voter': 'Name'})

# Remove that time an empty chair voted "Yes"
clus_vote_df[1] = clus_vote_df[1][clus_vote_df[1].Name != '. .']

It's possible there were multiple identical votes on the same day, but there's no good way to tell if they are data input problems or real votes. Even if they are real votes, it's still probably okay to drop since identical votes are  not going to provide much additional insight into our councillors.

In [ ]:
clus_vote_df[0] = clus_vote_df[0].drop_duplicates()
clus_vote_df[1] = clus_vote_df[1].drop_duplicates()

Our data has one voter's vote per row. We want to cluster councillors based on how often they vote together. This means we want our data to represent the distribution of voters responses each time they voted. We first need to identify each individual resolution that the voters were asked to vote upon, add that column back into our original dataframe, then perform a pivot.

In [ ]:
pivot_df = {}
for city_id, df in clus_vote_df.items():
    print(f'City = {city_dict[city_id]}')

    # All the information about the resolution, not the voter
    col_list = df.columns.drop(['Name','Vote']).to_list()

    # This maddening line collects all the resolution-based information and assigns a "vote_id" to each
    temp_df = df.groupby(col_list).size().reset_index().drop(columns=0).reset_index().rename(columns={'index':'vote_id'})
    
    # We merge this back into the original dataframe so we have a new "vote_id" column
    merge_df = pd.merge(left = temp_df, right = df, on = col_list, how='left')
    
    # To get ready to pivot, we only need the name, vote, and vote_id
    # We also need there to be no duplicates, but somehow we have ~100 in the Toronto data? This is a bug I haven't resolved
    merge_df = merge_df.drop(columns=col_list).drop_duplicates(subset=['vote_id','Name'])

    # Finally pivot
    pivot_df[city_id] = merge_df.pivot(index='vote_id', columns='Name', values='Vote')
    
    # And replace absences with NaNs since we're done evaluating attendance
    pivot_df[city_id] = pivot_df[city_id].replace('Absent',np.NaN)

    display(pivot_df[city_id])

### Clustering

Now we finally have our data in a format where we can compare how similar two councillors are based upon how frequently they vote together. To do this in a sensible way, we should make a model about how councillors vote.

One possible place to start is to assume that each councillor's response on each vote is essentially a Bernoulli trial with some probability $p$ of voting `Yes` and $1-p$ of voting `No`. These probabilities would vary wildly vote to vote, so it's very unreasonable to turn this directly into some sort of Binomial and make generalized statements about voting behaviour.

However, when we look at two councillors, there is a different Bernoulli trial occurring... these two councillors will give the same response to a vote with some probability $p$ and different responses with $1-p$. We've seen that some motions are more likely to produce disagreements than others, but over a large enough sample size, it's not unreasonable to say that these probabilities of (dis)agreement are fairly constant between those two councillors. Suddenly, with a few caveats, we can arrive at a Binomial!

Specifically, if councillors $A$ and $B$ are both voting in $n_{AB}$ votes, they'll disagree $d_{AB}$ times with a probability of disagreeing $D_{AB}$.
$$d_{AB} \sim \text{Binomial}(n_{AB},D_{AB})$$

Before we get ahead of ourselves, what are those caveats?

1. $D_{AB}$ is assumed to be independent of vote, which really only holds if each pair of councillors get a similar distribution of motions to vote upon and enough votes to let $D_{AB}$ truly represent some kind of average disagreement probability
2. Councillors are assumed to be static and unchanging over the time they spend in office.
3. Votes are assumed to be independent of one another. 

There are more assumptions we could acknowledge, but these three seem the most dubious. If we ever wanted to return to this project and improve our confidence in the result, tackling caveat #1 by controlling the motion types represented in our data would be a good starting point!

From our data, $n_{AB}$ and $d_{AB}$ are numbers we can directly compute. $D_{AB}$ however needs to be estimated. We could simply use the maximum likelihood estimator (MLE):

$$\hat{D}_{AB} = \frac{d_{AB}}{n_{AB}}$$

There are two issues with this. Firstly, not all estimates of $D_{AB}$ will be equal since some pairs of councillors will have thousands of votes and some will only have a few (represented by $n_{AB}$), and it would be good to differentiate these situations. Secondly, we will have some pairs of councillors that agreed every (or almost every) time they were able to vote together... this usually happens when $n_{AB}$ is very low, but that doesn't mean we genuinely believe $D_{AB}$ should be zero.

A bayesian approach tackles both these problems simultaneously. As is typical with estimating Binomial probabilities, we'll use the Beta distribution as a conjugate prior for $D_{AB}$,

$$D_{AB} \sim \text{Beta}(\alpha,\beta)$$

Now we've just pushed our problem onto estimating $\alpha$ and $\beta$. Bayesian inference gives us a means for updating our estimates for these two (hyper/meta)parameters as we acquire more data, specifically:

$$\alpha' = \alpha + d_{AB}$$
$$\beta' = \beta + (n_{AB}-d_{AB})$$

We need to know the values of $\alpha$ and $\beta$ before we have any data, or our prior, and for simplicity's sake we can choose the uniform prior of $\alpha = \beta = 1$. Using a different prior (like Jeffrey's) is not a bad idea, but hopefully with large enough $n_{AB}$ this shouldn't matter. Our final estimator becomes:

$$\hat{D}_{AB} = \frac{\alpha'}{\alpha'+\beta'}$$

$$= \frac{d_{AB} + 1}{n_{AB} + 2}$$

$$\sigma^2_{\hat{D}_{AB}} = \frac{(d_{AB} + 1)(n_{AB}-d_{AB}+1)}{(n_{AB} + 2)^2(n_{AB}+3)}$$ 

At long last, we have the elements we need for a dissimilarity matrix for our councillors. We can use $D_{ij}$ as our dissimilarity value and $w_{ij}=\frac{1}{\sigma_{ij}^2}$ as a weighting.

One final element to consider is that when $n_{AB}$ is very small, we run into a number of assumptions failing, so we'll be sure to remove elements when this is the case.

In [ ]:
#first get matrix for d_ij and n_ij, then use those to compute D_ij and w_ij
d_df = {}
n_df = {}

n_cutoff = 100

for city_id, df in pivot_df.items():
    # Unfortunately I can't think of a clean way to do this in pandas without iterating through the columns, which is slower.
    # At least we can resolve only half the matrix and rely on symmetry to get the other half!
    all_cols = df.columns
    rem_cols = df.columns
    d_df[city_id] = pd.DataFrame(columns = all_cols, index = all_cols, dtype = int)
    for col1 in all_cols:
        rem_cols = rem_cols.drop(col1)
        # Doing this should ensure we have every unordered combination of col1, col2
        for col2 in rem_cols:
            # Disagreements are when both voters voted (not null) but voted differently
            d_df[city_id].at[col1,col2] = len(df.loc[df[col1].notnull() & df[col2].notnull() & (df[col1] != df[col2])])
    
    # reset rem_cols so we can loop through again 
    rem_cols = df.columns
    n_df[city_id] = pd.DataFrame(columns = all_cols, index = all_cols, dtype = int)
    for col1 in all_cols:
        rem_cols = rem_cols.drop(col1)
        for col2 in rem_cols:
            val = len(df.loc[df[col1].notnull() & df[col2].notnull()])
            # remove entries where n is less than the cutoff
            n_df[city_id].at[col1,col2] = val if val >= n_cutoff else np.NaN

    n_df[city_id] = n_df[city_id].replace(0,np.NaN)

Since $D_{ij}$ and $w_{ij}$ can be performed element-wise, they are much simpler and easier to compute!

In [ ]:
D_df = {}
w_df = {}

for city_id in [0,1]:
    print(f'City = {city_dict[city_id]}')
    
    D_df[city_id] = (d_df[city_id]+1)/(n_df[city_id]+2)
    w_df[city_id] = 1/((d_df[city_id]+1)*(n_df[city_id]-d_df[city_id]+1)/((n_df[city_id]+2)**2*(n_df[city_id]+3)))
    
    display(D_df[city_id])
    display(w_df[city_id])

Despite that these matrices are symmetric, so we can fill in the bottom-left values with the top-right values, these matrices are pretty sparse! This is because plenty of councillors never served with other councillors on this list, and so we can't directly state how similar or dissimilar they are. 

I did my fair share of googling clustering algorithms for sparse boolean non-euclidean data with weights, but found nothing satisfying. Imputting our missing values with something meaningful would be helpful, but that's not intuitive.

Still, this is supposed to be a *fun* project, and to me, inventing a clustering algorithm from scratch actually does seem ***fun***!

To make the clustering easier, it would be really nice if we could fill in the missing entries in our dissimilarity matrix. If we know $D_{AB}$ and $D_{BC}$ but not $D_{AC}$, is there anything we can say about it anyway? And can we bias our results so that our clustering will prefer to group councillors who coexisted simultaneously over their hypothetical time-travelling colleagues?

One idea that comes to mind comes from looking at the probabilities of agreement, or $A_{ij} = 1-D{ij}$. Specifically the probability of A and C agreeing would be connected to the probability of A agreeing with B, and B agreeing with C. In math:

$$A_{AC}=A_{AB}A_{BC}$$
$$D_{AC}=1-((1-D_{AB})(1-D_{BC})$$

With the weights being the inverse variances, we should probably find $w_{AC}$ too while we're at it. Using the *actually not very great* assumption that these are gaussian variances, we can propagate our uncertainty as follows:

$$\sigma_{AC}^2 = D_{AC}^2 \left[\frac{\sigma_{AB}^2}{(1-D_{AB})^2}+\frac{\sigma_{AC}^2}{(1-D_{AC})^2}\right]$$

Of course since these are probabilities, $A,D\in[0,1]$, which means that the following relations are true:

$$A_{AC} < A_{AB}, A_{BC}$$
$$D_{AC} > D_{AB}, D_{BC}$$

But that's actually a good thing! If $D_{ij}$ is something to minimize in a cluster, then we want councillors to group with their co-existing colleagues first!

One final wrinkle arrives in the very likely event that councillors A and Z never vote together, but there are many B,C,D,... councillors that vote with both A and Z. How do we handle this? Since we already know how to calculate every path for $D_{AZ}$ along with every associated uncertainty, and we know these values are all bigger than their intermediate steps, we can  just find the mean of all these paths, weighted by their individual inverse variances! When taking the variance for this average, we have to remember to grab the weighted variance of the paths, not the weighted variance of the mean.

While there's no single correct answer for an imputation like this and there may be simpler options or more rigourous options, this choice seems to balance both in my mind and also winds up giving good results!

In [ ]:
# Write out some functions that we'll use a lot of
def weightedVariance(input_vals, sd=False):
    '''
    vals = [x[0] for x in input_vals]
    #if sd:
        #weights = [(1/(x[1])**2) for x in input_vals]
    #else:
    weights = [x[1] for x in input_vals]
    mean = weightedMean(input_vals, sd)
    num = 0
    denom = 0
    '''
    mean = weightedMean(input_vals)
    num = 0
    denom = 0
    for v,w in input_vals:#zip(vals, weights):
        num += w*((v-mean)**2)
        denom += w
    return num/denom
        
def weightedMean(input_vals, sd=False):
    '''
    weights = []
    vals = [x[0] for x in input_vals]

    #if sd:
    #    weights = [(1/(x[1])**2) for x in input_vals]
    #else:
    weights = [x[1] for x in input_vals]
    '''
    
    num = 0
    denom = 0

    for v,w in input_vals:#zip(vals, weights):
        num += v*w
        denom += w
        
    return num/denom
    


This next function is probably the biggest user of processing time in this whole project. For lists of councillors any bigger, this would be the first target to be migrated to C or to be rewritten in some other fashion. 

The idea is to find all the missing values in the table, then use the connections between the councillors to fill in as much as we can. It's possible there are councillors with no other councillors connecting them, but that's okay, once we've filled in our missings once, we are far more likely to have just imputed a value that will help us make a new connection!

It's also possible that there is some set of councillors who have never voted with another set of councillors, and that no connecting councillor exist. In that case, this will fail, but so too will our entire clustering process. This seems a very unlikely situation however, so it falls in the category of "problems that might not actually be problems".

In [ ]:
def imputeMissings(clus_df, wclus_df):
    # this function alters clus_df and wclus_df!
    
    i=0
    while clus_df.isna().sum().sum() > 0:
        print(f'in while loop, i={i}')
        i+=1
        
        # find array locations of all missing elements
        nans = np.where(clus_df != clus_df)
        print(f'--There are {len(nans[0])} missing elements')
        
        j=0
        #iterate through missings and find where the two unconnected nodes have a node in common
        for nans_row, nans_col in zip(nans[0],nans[1]):
            if j%100 == 0:
                print(f'processed {j} missing elements')
            j+=1
            if clus_df.iat[nans_row,nans_col] == clus_df.iat[nans_row,nans_col]:
                #this can happen because of the symmetrization
                continue
            nan_row_options = clus_df.iloc[nans_row].dropna().index
            nan_col_options = clus_df.iloc[:,nans_col].dropna().index
            connections = np.intersect1d(nan_row_options,nan_col_options)
                        
            # if no nodes in common, move on. Eventually we should get every NaN unless data unclusterable somehow
            if len(connections)==0:
                continue

            # if there are multiple common nodes, we want to loop over them and take an average
            # the agreement value for each common node connection is a product of the agreements
            vals = []
            for c in connections:
                d_ab = clus_df.iloc[nans_row][c]
                d_bc = clus_df.iloc[:,nans_col][c]
                d_ac = 1-(1-d_ab)*(1-d_bc)
                w_ab = wclus_df.iloc[nans_row][c]
                w_bc = wclus_df.iloc[:,nans_col][c]
                w_ac = 1/(d_ac**2*(1/((1-d_ab)**2*w_ab)+1/((1-d_bc)**2*w_bc)))
                vals.append((d_ac,w_ac))
            val = weightedMean(vals)
            # We want the stdev of the distribution, not of the mean
            w_val = 1/weightedVariance(vals)
            
            #fill in our dataframes with the results, don't forget to keep symmetric!
            clus_df.iat[nans_row,nans_col] = val
            clus_df.iat[nans_col,nans_row] = val
            wclus_df.iat[nans_row,nans_col] = w_val
            wclus_df.iat[nans_col,nans_row] = w_val    
    return

Now that we've defined our functions, let's actually do the imputation!

In [ ]:
# Make a directory to store the imputed data so we don't have to redo this step too often.
os.makedirs('intermediates', exist_ok=True) 

# We'll keep our original D_df and w_df just in case
D_imp_df = {}
w_imp_df = {}

for city_id in [0,1]:
    print(f'City = {city_dict[city_id]}')

    # For simplicity, we'll assign a number to each councillor
    D_imp_df[city_id] = D_df[city_id].copy()
    D_imp_df[city_id].index = range(0,len(D_imp_df[city_id].columns))
    D_imp_df[city_id].columns = range(0,len(D_imp_df[city_id].columns))
    w_imp_df[city_id] = w_df[city_id].copy()
    w_imp_df[city_id].index = range(0,len(w_imp_df[city_id].columns))
    w_imp_df[city_id].columns = range(0,len(w_imp_df[city_id].columns))


    # Make symmetric 
    rem_cols = D_imp_df[city_id].columns.copy()
    for col1 in D_imp_df[city_id].columns:
        rem_cols.drop(col1)
        for col2 in rem_cols:
            D_imp_df[city_id].at[col2,col1] = D_imp_df[city_id].at[col1,col2]
            w_imp_df[city_id].at[col2,col1] = w_imp_df[city_id].at[col1,col2]

    # Remove councillors with no entries. This happens because of our n_ij > n_cutoff we did earlier
    drop_cols = D_imp_df[city_id].columns[D_imp_df[city_id].notna().any() == False]
    print(f'The following councillors have been dropped due to low n:{drop_cols.to_list()}')
    D_imp_df[city_id] = D_imp_df[city_id].drop(drop_cols, axis = 0).drop(drop_cols, axis = 1)
    w_imp_df[city_id] = w_imp_df[city_id].drop(drop_cols, axis = 0).drop(drop_cols, axis = 1)

    # Set trace to 0
    for col1 in D_imp_df[city_id].columns:
        D_imp_df[city_id].at[col1,col1] = 0


    imputeMissings(D_imp_df[city_id], w_imp_df[city_id])

    display(D_imp_df[city_id].head())
    display(w_imp_df[city_id].head())

    # Save to file to save time if we have to redo this later
    D_imp_df[city_id].to_csv(f'intermediates/D_imp_df{city_id}.csv')
    w_imp_df[city_id].to_csv(f'intermediates/w_imp_df{city_id}.csv') 

Next we need some metric evaluate how good a cluster is. Our dissimilarity matrix already gives us a value for how dissimilar a cluster of any 2 councillors are, and if we ditch the idea that the dissimiliarity has anything to do with probability of voting, we can interpret it as a type of distance, and its square as a type of variance.

Clustering based on variance is a very common metric (see k-means clustering), but there is a catch with our data. Not actually being a distance, it's non-euclidean, meaning that there's no natural physical incentive for our data points to spread out in the same way that points on a map might be spread. This leads our clustering to favour having one large group with all councillors in one spot, and several tiny one-councillor sized clusters. We can "fix" this by optimizing the variance multiplied by the number of points in a cluster, thus biasing our clusters to be equally sized.

In [ ]:
# Cluster variance based on councillors in cluster
def getClusVar(cols, D_df, w_df):
    if len(cols) == 1:
        return 0
    
    num = 0
    denom = 0
    
    rem_cols = cols.copy()
    for col1 in cols:
        rem_cols.remove(col1)
        for col2 in rem_cols:
            v = D_df.at[col1,col2]
            w = w_df.at[col1,col2]
            num += w*v**2
            denom += w
            
    if denom != denom:
        return np.NaN
    return num/denom

# Cluster variance multiplied by size of cluster (via the weights of the councillors in the cluster)
def getClusNVar(cols, D_df, w_df):
    if len(cols) == 1:
        return 0
    
    num = 0
    
    rem_cols = cols.copy()
    for col1 in cols:
        rem_cols.remove(col1)
        for col2 in rem_cols:
            v = D_df.at[col1,col2]
            w = w_df.at[col1,col2]
            num += w*v**2

    return num

In [ ]:
# agglomerative clustering based on objective function ~= variance * cluster size

# This keeps track of which councillors are in which cluster
clusters_id = {}

# This keeps track of how the total objective function changes with the number of clusters
nClus_to_obj_func = {}

# This keeps track of which clusters existed at what points
clus_timeline = {}

# This keeps track of councillor_id numbers and councillor names
id_to_name = {}

for city_id in [0,1]:
    print(f'City = {city_dict[city_id]}')

    # We need to know who the councillors are
    councillor_list = D_df[city_id].columns.to_list()
    id_to_name[city_id] = dict(zip(range(0,len(councillor_list)),councillor_list))

    # Clus_df has the rows and columns of clusters, for now each with just a single councillor
    # The non-diagonal elements represent how much the to-be-minimized objective function grows with each merging of clusters
    clus_df = D_imp_df[city_id]**2
    
    # Unfortunately we have to set the diagonal to NaN so we can take the min later and not grab the diagonal values
    for col1 in clus_df.columns:
        clus_df.at[col1,col1]=np.NaN

    # Make initial clusters of 1 councillor per cluster
    # Clusters_id[city_id] is of the form {cluster_id : ([list of councillor_ids],N*Variance)}
    clusters_id[city_id] = {}
    for col in clus_df.columns:
        clusters_id[city_id].update({col:([col],0)})

    # Initialize these before clustering
    nClus_to_obj_func[city_id] = {}
    clus_timeline[city_id] = {}

    # This is the objective function with the current number of clusters
    nClus_to_unexp[city_id][len(clus_df.columns)] = 0
    
    # This is the current state of the clusters
    clus_timeline[city_id][len(clus_df.columns)] = clus_df.columns

    # Keep merging until only 2 clusters left
    while len(clus_df.columns)>2:
        # Find the best two clusters to merge
        min_row = clus_df.min().idxmin()
        if min_row!=min_row:
            # It is possible our data has only unmergeable clusters remaining. It's not common though.
            break
        min_col = clus_df[min_row].idxmin()
        if min_col!=min_col:
            break

        #make a new cluster from merging the best two
        new_cluster_elems = clusters_id[city_id][min_row][0] + clusters_id[city_id][min_col][0]
        obj_func = clus_df.at[min_row,min_col]
        new_clus_id = clus_df.columns.max() + 1
        clusters_id[city_id].update({new_clus_id:(new_cluster_elems, obj_func)})

        # some debugging output
        #print(f'min_row={min_row}, min_col={min_col}')
        #print(f'{clusters_id[city_id][min_row][0]}+{clusters_id[city_id][min_col][0]}={new_cluster_elems} with {strain},{clus_df.min().min()}')

        #drop the clusters that are merging from clus_df
        clus_df = clus_df.drop(min_row, axis=0).drop(min_row, axis=1).drop(min_col, axis=0).drop(min_col, axis=1)

        #add the new cluster to clus_df
        new_row = [getClusNVar(new_cluster_elems+clusters_id[city_id][clus_id][0],D_imp_df[city_id],w_imp_df[city_id]) for clus_id in clus_df.columns.to_list()]
        clus_df.loc[new_clus_id] = new_row
        #now add the column, with the last element being the diagonal, so therefore NaN
        new_row.append(np.NaN)
        clus_df[new_clus_id] = new_row

        #update our metrics
        nClus_to_obj_func[city_id][len(clus_df.columns)] = np.sum([clusters_id[city_id][clus_id][1] for clus_id in clus_df.columns])
        clus_timeline[city_id][len(clus_df.columns)] = clus_df.columns

print('Done!')

In [ ]:
for city_id in [0,1]:
    print(f'City = {city_dict[city_id]}')
    plt.figure()
    plt.plot(nClus_to_obj_func[city_id].keys(),nClus_to_obj_func[city_id].values())

There's no really good way to say what the best number of clusters is, but the "elbow method" looks for where the above plots curve the most. That seems to be with about 10 clusters for Toronto, and 5 for Calgary. Let's see what these clusters look like!

In [ ]:
def getNamesInClusters(clus_ids, clusters, id_to_name):
    cur_clusters = [clusters[clus_id][0] for clus_id in clus_ids]
    return [[id_to_name[x] for x in clus] for clus in cur_clusters]

# Toronto
print(f'City = {city_dict[0]}')
final_clusters = getNamesInClusters(clus_timeline[0][10],clusters_id[0],id_to_name[0])
for i, clus in enumerate(final_clusters):
    string = ''
    for name in clus:
        if name is clus[-1]:
            string += name
        else:
            string += name + ', '
    
    print(i+1,string)
    
print()

# Calgary
print(f'City = {city_dict[1]}')
final_clusters = getNamesInClusters(clus_timeline[1][5],clusters_id[1],id_to_name[1])
for i, clus in enumerate(final_clusters):
    string = ''
    for name in clus:
        if name is clus[-1]:
            string += name
        else:
            string += name + ', '
    
    print(i+1,string)
print()

So there we have it! I like to see Fords and Holydays generally placed together, and Tory with McKelvie also matches expectations. I'm a lot less knowledgeable about Calgary politics, but my brother who works at the city hall there says "That's largely correct. I'd accept that sorting". High praise indeed!